In [1]:
import pandas as pd
import numpy as np

## If numpy and pyomo return an error regarding np.float_ and np.float64, uncomment the aline bellow (this error should be resolved, but did occur occasionally due to mismatched versions of the numpy and pyomo packages)
#np.float_ = np.float64

import pyomo.environ as pe
import pyomo.opt as po
import time

### Pulling Load Data

In [2]:
se_loads_residential= pd.read_csv(
    'Data/LoadProfiles_SE_Residential.csv',
    sep= ','
)



rows_to_add= (se_loads_residential[(8760 - 24): 8760]).reset_index(drop= True)
new_hours= list(range(8761, (8761+24), 1))
rows_to_add['Hour']= new_hours



se_loads_residential= pd.concat(
    [
        se_loads_residential,
        rows_to_add
    ]
)
se_loads_residential= se_loads_residential.reset_index(drop= True)

In [5]:
se_loads_industrial= pd.read_csv(
    'Data/LoadProfiles_SE_Industrial.csv',
    sep= ','
)



rows_to_add= (se_loads_industrial[(8760 - 24): 8760]).reset_index(drop= True)
new_hours= list(range(8761, (8761+24), 1))
rows_to_add['Hour']= new_hours



se_loads_industrial= pd.concat(
    [
        se_loads_industrial,
        rows_to_add
    ]
)
se_loads_industrial= se_loads_industrial.reset_index(drop= True)

In [6]:
loads_residential= [
    'residential1',
    'residential2',
    'residential3',
    'residential4',
    'residential5'
]

loads_industrial= [
    'industrial1',
    'industrial2'
]



## Limiting for PaperA supplementation batch
loads_residential= [
    #'residential1',
    #'residential2',
    #'residential3',
    'residential4',
    #'residential5'
]

loads_industrial= [
    'industrial1',
    'industrial2'
]

### Pulling Regional Data

In [7]:
data_de=    pd.read_csv('Data/data_de.csv', sep= ';', index_col= False)
data_es=    pd.read_csv('Data/data_es.csv', sep= ';', index_col= False)
data_it=    pd.read_csv('Data/data_it.csv', sep= ';', index_col= False)
data_pl=    pd.read_csv('Data/data_pl.csv', sep= ';', index_col= False)
data_se=    pd.read_csv('Data/data_se.csv', sep= ';', index_col= False)
data_se1=   pd.read_csv('Data/data_se1.csv', sep= ';', index_col= False)
data_se2=   pd.read_csv('Data/data_se2.csv', sep= ';', index_col= False)
data_se3=   pd.read_csv('Data/data_se3.csv', sep= ';', index_col= False)
data_se4=   pd.read_csv('Data/data_se4.csv', sep= ';', index_col= False)



data_dict= {
    'DE': data_de,
    'ES': data_es,
    'IT': data_it,
    'PL': data_pl,
    'SE': data_se,
    'SE1': data_se1,
    'SE2': data_se2,
    'SE3': data_se3,
    'SE4': data_se4,
}

regions= [
    'DE',
    'ES',
    'IT',
    'PL',
    'SE',
    'SE1',
    'SE2',
    'SE3',
    'SE4',
]


## Limiting for PaperA supplementation batch
regions= [
    'IT',
    'PL',
    'SE3'
]

### Setting Resulting Dataframe & Decision Variable Ranges

Adjust ranges below to limit the amount of configuration scenarios

In [9]:
solar_multiples=    [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
bess_multiples=     [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
bess_capacities=    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
optimise_for=       ['Price', 'CO_2_eq']

In [10]:
## Define the sets/boundaries of the flow/hour table for our values
flows_global= [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
]

hours_global=   set(range(8760 + 24))
time_window=    48
num_iterations= (8760/time_window)

## Grid availability for each hour is set to 1000 MW (== 1 000 000 kW), should be more than enough for our model demand and any potential BESS demand for energy arbitrage
grid_production= 1000000

Function to check for existing result profiles (if iteratively running large batches)

In [11]:
def file_exists(filename):
    try:
        # Try to open the file in read mode
        with open(filename, 'r'):
            pass
    except FileNotFoundError:
        return False
    return True

### Optimiser
*OBS!: Create a subdirectory (folder) titled 'RESULTS' before running

In [13]:
### Industrial Loads
## Comment this out (turn it off) if errors occur with pandas or with saved results, then troubleshoot
pd.options.mode.chained_assignment = None  # Default is 'warn'
### Some comment warning of Pandas 3.0 might persist, will be fixed soon


flows_results= [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
    'SoC'
]
#results= pd.DataFrame(index= hours_global, columns= flows_results)

## Other BESS parameters
efficiency_charge=      0.98
efficiency_discharge=   0.96
efficiency_inverter=    0.97


## what multiple of the load we will limit the grid injection by.
## Default is 20%
overreach_capacity = 20
demand_multiple= (1 + overreach_capacity/100)
#demand_multiple= (float(data['Load'].max())) * (1 + overreach_capacity/100)



for bess_capacity in bess_capacities:
    for solar_multiple in solar_multiples:
        for bess_multiple in bess_multiples:
            for load_profile in loads_industrial:
                for curve in optimise_for:
                    for region in regions:
                        if not file_exists(f'RESULTS/{curve}_{region}_{load_profile}_{solar_multiple}pv_{bess_multiple}bess_{bess_capacity}hr.csv'):
                            results= pd.DataFrame(index= list(hours_global), columns= flows_results)

                            data= data_dict[region]
                            load_data= se_loads_industrial[load_profile]
                            
                            ## In kW, solar multiple multiplied by maxLoad (== 3000 kW) and divided by base PV production (100MW == 100 000 kW)
                            #p_solar=    (solar_multiple * data['Load'].max())/100000
                            #p_solar=    (solar_multiple * 1) * 1000 / 100000
                            ## In kW; As input solar production is normalised to 1000 kW, and the load is normalised to 1 MW, we just multiply with the solar multiple
                            p_solar=    solar_multiple
                            ## In kW, bess multiple multiplied by maxLoad (== 3000 kW)
                            #p_bess=     bess_multiple * data['Load'].max()
                            ## In kW; we need to multiply the base multiple (which is in MW) with 1000
                            p_bess=     bess_multiple * 1000
                            ## In hours
                            t_bess=     bess_capacity
                            if t_bess != 0:
                                p_bess= p_bess
                            else:
                                p_bess= 0
                            ## In kWh
                            e_bess=     p_bess * t_bess
                            soc_initial=e_bess * 0.5
                            start_time= time.time()
                            print(f'PV Multiple: {solar_multiple}x Peak Load [MW]|| BESS Multiple: {bess_multiple}x Peak Load [MW]; {bess_capacity} hr (at rated power) == {bess_multiple * bess_capacity} [MWh]')
                            print(f'Optimising Load: {load_profile} for {region} via the {curve} signal')
                            

                            for iteration in range(1, (int(num_iterations)) +1):

                                # print(f'P_PV: {p_solar * 100 * 1000} kW')
                                # print(f'P_BESS: {p_bess} kW')
                                # print(f'T_BESS: {t_bess} hr')
                                # print(f'Done: {round((iteration/num_iterations) * 100, 0)}%')


                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Base data imports for the model
                                hours=  list(range(((iteration - 1)*time_window), ((iteration)*time_window)))

                                demand= {}
                                pv_production= {}
                                for hour in hours:
                                    demand[hour]= load_data[hour] * 1000
                                    pv_production[hour]= (data['solar_pv'][hour] * p_solar)

                                costs_keys= []
                                for flow in flows_global:
                                    for hour in hours:
                                        costs_keys.append((flow, hour))


                                costs_economic= {}
                                for i in range(len(costs_keys)):
                                    if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                                        costs_economic[costs_keys[i]]= (data['Price'][costs_keys[i][1]]/1000)
                                    elif costs_keys[i][0] == 'P_PV_to_Grid':
                                        costs_economic[costs_keys[i]]= (-1) * ((data['Price'][costs_keys[i][1]])/1000)
                                    elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
                                        costs_economic[costs_keys[i]]= 0
                                    elif costs_keys[i][0] =='P_BESS_to_Grid':
                                        costs_economic[costs_keys[i]]= (-1) * ((data['Price'][costs_keys[i][1]])/1000)
                                    elif costs_keys[i][0] == 'P_BESS_to_Load':
                                        costs_economic[costs_keys[i]]= 0
                                    elif costs_keys[i][0] == 'P_PV_curtailment':
                                        costs_economic[costs_keys[i]]= 1000/1000
                                    else:
                                        continue
                                costs_environmental= {}
                                for i in range(len(costs_keys)):
                                    if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                                        costs_environmental[costs_keys[i]]= data['CO_2_eq'][costs_keys[i][1]]
                                    elif costs_keys[i][0] == 'P_PV_to_Grid':
                                        costs_environmental[costs_keys[i]]= (-1) * ((data['CO_2_eq'][costs_keys[i][1]]))
                                    elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
                                        costs_environmental[costs_keys[i]]= 0
                                    elif costs_keys[i][0] =='P_BESS_to_Grid':
                                        costs_environmental[costs_keys[i]]= (-1) * ((data['CO_2_eq'][costs_keys[i][1]]))
                                    elif costs_keys[i][0] == 'P_BESS_to_Load':
                                        costs_environmental[costs_keys[i]]= 0
                                    elif costs_keys[i][0] == 'P_PV_curtailment':
                                        costs_environmental[costs_keys[i]]= 1000/1000
                                    else:
                                        continue

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Initialize model
                                model= pe.ConcreteModel()
                                ## Sets
                                model.flows= pe.Set(initialize= flows_global, ordered= True)
                                model.hours= pe.Set(initialize= hours, ordered= True)

                                ## Parameters
                                model.grid_production=      pe.Param(initialize= grid_production)
                                model.demand=               pe.Param(model.hours, initialize= demand)
                                model.pv_production=        pe.Param(model.hours, initialize= pv_production)

                                model.Efficiency_charge=    pe.Param(model.hours, initialize= efficiency_charge)
                                model.Efficiency_inverter=  pe.Param(model.hours, initialize= efficiency_discharge)
                                model.Efficiency_discharge= pe.Param(model.hours, initialize= efficiency_inverter)

                                model.SoCmin=               pe.Param(initialize= (e_bess*0.1))
                                model.SoCmax=               pe.Param(initialize= (e_bess*0.9))

                                model.MaxCharge=            pe.Param(initialize= p_bess)

                                ## OBS!: Read the last value of SoC, OR initial if first day
                                if iteration == 1:
                                    model.SoCinitial=       pe.Param(initialize= soc_initial)
                                #model.SoCinitial=       pe.Param(initialize= soc_initial)
                                else:
                                    if pd.isna(results['SoC'].iloc[len(results) -1]):# == True:
                                        model.SoCinitial=       pe.Param(initialize= (e_bess*0.1))
                                    else:
                                        last_SoC_value=         float(results['SoC'].iloc[-1])
                                        model.SoCinitial=       pe.Param(initialize= last_SoC_value)
                                
                                
                                #model.arbitrageLimit=       pe.Param(model.hours, initialize= (p_solar * 1000000))
                                model.arbitrageLimit=       pe.Param(model.hours, initialize= demand_multiple)
                                if curve == 'Price':
                                    model.costs=            pe.Param(model.flows, model.hours, initialize=costs_economic, default= 1)
                                elif curve == 'CO_2_eq':
                                    model.costs=            pe.Param(model.flows, model.hours, initialize=costs_environmental, default= 1)

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Variables
                                # Variable for the power flows in our system
                                model.p=            pe.Var(model.flows, model.hours, domain= pe.NonNegativeReals)
                                # Variable to track the State-of-Charge in the battery
                                model.SoC=          pe.Var(model.hours, domain= pe.Reals, bounds= (model.SoCmin, model.SoCmax))
                                # Two variables to track/limit the charge/discharge operation at the same time (ref: doi.org/10.2991/978-94-6463-156-2_13)
                                model.Sw_charge=    pe.Var(model.hours, domain= pe.Binary)
                                model.Sw_discharge= pe.Var(model.hours, domain= pe.Binary)

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Objective
                                minCosts = sum(model.p[f, t] * model.costs[f, t] for f in model.flows for t in model.hours)
                                model.objective= pe.Objective(sense= pe.minimize, expr= minCosts)

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Constraints - Declaration
                                
                                # Load Fullfilment from the flows _to_Load in the system 
                                loadFullfilment=    {t: (model.p['P_PV_to_Load', t] * efficiency_inverter + model.p['P_BESS_to_Load', t] * (efficiency_discharge * efficiency_inverter) + model.p['P_Grid_to_Load', t]) == model.demand[t] for t in model.hours}

                                # Allocation of PV production as a resource
                                pvProduction=       {t: (model.p['P_PV_to_Load', t] + model.p['P_PV_to_BESS', t] + model.p['P_PV_to_Grid', t] + model.p['P_PV_curtailment', t]) == model.pv_production[t] for t in model.hours}
                                
                                # Load multiple limits the arbitrage flow
                                arbitrageFlow=      {t: (model.p['P_BESS_to_Grid', t] * (efficiency_discharge * efficiency_inverter) + model.p['P_PV_to_Grid', t] * efficiency_inverter) <= model.arbitrageLimit[t] for t in model.hours}
                                
                                # Limit to grid connection - for the purposes at this point it 1 GW (meaning multiple times higher than our system, representing an "unlimited" amount of power we can draw from the grid)
                                gridFlow=           {t: (model.p['P_Grid_to_Load', t] + model.p['P_Grid_to_BESS', t]) <= model.grid_production for t in model.hours}
                                
                                # Constraint to limit maximum charging capacity
                                bessChargeFlow=     {t: (model.p['P_Grid_to_BESS', t] + model.p['P_PV_to_BESS', t]) <= model.Sw_charge[t] * model.MaxCharge for t in model.hours}
                                bessDischargeFlow=  {t: (model.p['P_BESS_to_Grid', t] + model.p['P_BESS_to_Load', t]) <= model.Sw_discharge[t] * model.MaxCharge for t in model.hours}
                                
                                # Constraint to limit charging/discharging in the same time block
                                # This can be implemented by adjusting the bessChargeFlow and bessDischargeFlow (see: doi.org/10.2991/978-94-6463-156-2_13)
                                limitValidation=    {t: model.Sw_charge[t] + model.Sw_discharge[t] <= 1 for t in model.hours}


                                def storage_state(model, t):
                                            if t == model.hours.first():
                                                return model.SoC[t] == model.SoCinitial + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (efficiency_discharge * efficiency_inverter))
                                            else:
                                                return model.SoC[t] == model.SoC[t-1] + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (efficiency_discharge * efficiency_inverter))

                                ## Constraints - Implementation
                                model.demandRule=       pe.Constraint(model.hours, expr= loadFullfilment)
                                model.pvProductionRule= pe.Constraint(model.hours, expr= pvProduction)
                                model.arbitrageRule=    pe.Constraint(model.hours, expr= arbitrageFlow)
                                model.gridRule=         pe.Constraint(model.hours, expr= gridFlow)
                                model.chargeRule=       pe.Constraint(model.hours, expr= bessChargeFlow)
                                model.dischargeRule=    pe.Constraint(model.hours, expr= bessDischargeFlow)
                                #model.limitCharge=      pe.Constraint(model.hours, expr= limitCharge)
                                #model.limitDischarge=   pe.Constraint(model.hours, expr= limitDischarge)
                                model.limitValidation=  pe.Constraint(model.hours, expr= limitValidation)
                                model.charge_state=     pe.Constraint(model.hours, expr = storage_state)


                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Solve the model for the time window (iteration)
                                solver= po.SolverFactory('glpk')
                                ## What happened often is that the Long-Step Dual Simplex method is used, which uses an iterative approach to arrive at a "local globally-optimal" solution, which can take a very long time to compute
                                ## We will limit the maximum time for finding the optimal solution, based either on the time limit (in seconds) or the minimal optimality gap (in percentages)
                                ## The two lines below set the time limit to 18 seconds and the minimal optimality gap to 3% (or lower)
                                solver.options['tmlim']= 15
                                solver.options['mipgap']= 0.05
                                ## Set tee to True if you want to see what the solver is doing (for troubleshooting etc.)
                                modelresults= solver.solve(model, tee= False)





                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Results Handling
                                for flow in flows_results:
                                    for hour in hours:
                                        if flow == 'SoC':
                                            results['SoC'][hour]= model.SoC[hour].value
                                        else:
                                            results[flow][hour]= model.p[flow, hour].value
                                    results[flow] = results[flow].astype(float)
                                # results= results.reset_index()
                                # results= results.rename(columns={'index': 'Hour'})
                                
                                #print(type(float(results['SoC'].iloc[-1])))
                                #print(results['SoC'].iloc[len(results) -1])

                                results['sum_power_flows'] = results.P_PV_to_Load + results.P_BESS_to_Load + results.P_Grid_to_Load
                                results['sum_power_flows'] = results['sum_power_flows'].astype(float)

                                for param in model.component_data_objects(pe.Param, active= True):
                                    model.del_component(param)
                                for variable in model.component_data_objects(pe.Var, active= True):
                                    model.del_component(variable)
                                for cons in model.component_data_objects(pe.Constraint, active=True):
                                    model.del_component(cons)
                                for obj in model.component_data_objects(pe.Objective, active= True):
                                    model.del_component(obj)
                                #model.del_component(model.SoCinitial)
                                #model.del_component(model.charge_state)
                                model.del_component(model.hours)
                                model.del_component(model.flows)


                                
                            results= results.reset_index()
                            results= results.rename(columns={'index': 'Hour'})
                            results.to_csv(
                                f'RESULTS/{curve}_{region}_{load_profile}_{solar_multiple}pv_{bess_multiple}bess_{bess_capacity}hr.csv',
                                sep= ',',
                                index=False,
                                columns=None
                            )
                            # if curve == 'price':
                            #     results.to_csv(
                            #         f'Results/Batch5/CaseB/price_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv',
                            #         sep= ',',
                            #         index=False,
                            #         columns=None
                            #     )
                            # elif curve == 'co2':
                            #     results.to_csv(
                            #         f'Results/Batch5/CaseB/co2_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv',
                            #         sep= ',',
                            #         index=False,
                            #         columns=None
                            #     )
                            results = []
                            end_time= time.time()
                            execution_time= end_time - start_time

                            # Convert execution time to hours, minutes, and seconds
                            hours, remainder = divmod(execution_time, 3600)
                            minutes, seconds = divmod(remainder, 60)

                            # Print the execution time in the format hh:mm:ss
                            print("Execution time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
                            print('-' * 50)
                            #print(' ')
#results

In [14]:
### Residential Loads
## Comment this out (turn it off) if errors occur with pandas or with saved results, then troubleshoot
pd.options.mode.chained_assignment = None  # Default is 'warn'


flows_results= [
    'P_PV_to_Load',
    'P_PV_to_BESS',
    'P_PV_curtailment',
    'P_PV_to_Grid',
    'P_BESS_to_Load',
    'P_BESS_to_Grid',
    'P_Grid_to_Load',
    'P_Grid_to_BESS',
    'SoC'
]
#results= pd.DataFrame(index= hours_global, columns= flows_results)

## Other BESS parameters
efficiency_charge=      0.98
efficiency_discharge=   0.96
efficiency_inverter=    0.97


## what multiple of the load we will limit the grid injection by.
## Default is 20%
overreach_capacity = 20
demand_multiple= (1 + overreach_capacity/100)
#demand_multiple= (float(data['Load'].max())) * (1 + overreach_capacity/100)



for solar_multiple in solar_multiples:
    for bess_capacity in bess_capacities:
        for bess_multiple in bess_multiples:
            for load_profile in loads_residential:
                for curve in optimise_for:
                    for region in regions:
                        if not file_exists(f'RESULTS/{curve}_{region}_{load_profile}_{solar_multiple}pv_{bess_multiple}bess_{bess_capacity}hr.csv'):
                            results= pd.DataFrame(index= list(hours_global), columns= flows_results)

                            data= data_dict[region]
                            load_data= se_loads_residential[load_profile]
                            
                            ## In kW, solar multiple multiplied by maxLoad (== 3000 kW) and divided by base PV production (100MW == 100 000 kW)
                            #p_solar=    (solar_multiple * data['Load'].max())/100000
                            #p_solar=    (solar_multiple * 1) * 1000 / 100000
                            p_solar=    solar_multiple
                            ## In kW, bess multiple multiplied by maxLoad (== 3000 kW)
                            #p_bess=     bess_multiple * data['Load'].max()
                            p_bess=     bess_multiple * 1000
                            ## In hours
                            t_bess=     bess_capacity
                            if t_bess != 0:
                                p_bess= p_bess
                            else:
                                p_bess= 0
                            ## In kWh
                            e_bess=     p_bess * t_bess
                            soc_initial=e_bess * 0.5
                            start_time= time.time()
                            print(f'PV Multiple: {solar_multiple}x Peak Load [MW]|| BESS Multiple: {bess_multiple}x Peak Load [MW]; {bess_capacity} hr (at rated power) == {bess_multiple * bess_capacity} [MWh]')
                            print(f'Optimising Load: {load_profile} for {region} via the {curve} signal')
                            

                            for iteration in range(1, (int(num_iterations)) +1):

                                # print(f'P_PV: {p_solar * 100 * 1000} kW')
                                # print(f'P_BESS: {p_bess} kW')
                                # print(f'T_BESS: {t_bess} hr')
                                # print(f'Done: {round((iteration/num_iterations) * 100, 0)}%')


                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Base data imports for the model
                                hours=  list(range(((iteration - 1)*time_window), ((iteration)*time_window)))

                                demand= {}
                                pv_production= {}
                                for hour in hours:
                                    demand[hour]= load_data[hour] * 1000
                                    pv_production[hour]= (data['solar_pv'][hour] * p_solar)

                                costs_keys= []
                                for flow in flows_global:
                                    for hour in hours:
                                        costs_keys.append((flow, hour))


                                costs_economic= {}
                                for i in range(len(costs_keys)):
                                    if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                                        costs_economic[costs_keys[i]]= (data['Price'][costs_keys[i][1]]/1000)
                                    elif costs_keys[i][0] == 'P_PV_to_Grid':
                                        costs_economic[costs_keys[i]]= (-1) * ((data['Price'][costs_keys[i][1]])/1000)
                                    elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
                                        costs_economic[costs_keys[i]]= 0
                                    elif costs_keys[i][0] =='P_BESS_to_Grid':
                                        costs_economic[costs_keys[i]]= (-1) * ((data['Price'][costs_keys[i][1]])/1000)
                                    elif costs_keys[i][0] == 'P_BESS_to_Load':
                                        costs_economic[costs_keys[i]]= 0
                                    elif costs_keys[i][0] == 'P_PV_curtailment':
                                        costs_economic[costs_keys[i]]= 1000/1000
                                    else:
                                        continue
                                costs_environmental= {}
                                for i in range(len(costs_keys)):
                                    if costs_keys[i][0] in ['P_Grid_to_Load', 'P_Grid_to_BESS']:
                                        costs_environmental[costs_keys[i]]= data['CO_2_eq'][costs_keys[i][1]]
                                    elif costs_keys[i][0] == 'P_PV_to_Grid':
                                        costs_environmental[costs_keys[i]]= (-1) * ((data['CO_2_eq'][costs_keys[i][1]]))
                                    elif costs_keys[i][0] in ['P_PV_to_Load', 'P_PV_to_BESS']:
                                        costs_environmental[costs_keys[i]]= 0
                                    elif costs_keys[i][0] =='P_BESS_to_Grid':
                                        costs_environmental[costs_keys[i]]= (-1) * ((data['CO_2_eq'][costs_keys[i][1]]))
                                    elif costs_keys[i][0] == 'P_BESS_to_Load':
                                        costs_environmental[costs_keys[i]]= 0
                                    elif costs_keys[i][0] == 'P_PV_curtailment':
                                        costs_environmental[costs_keys[i]]= 1000/1000
                                    else:
                                        continue

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Initialize model
                                model= pe.ConcreteModel()
                                ## Sets
                                model.flows= pe.Set(initialize= flows_global, ordered= True)
                                model.hours= pe.Set(initialize= hours, ordered= True)

                                ## Parameters
                                model.grid_production=      pe.Param(initialize= grid_production)
                                model.demand=               pe.Param(model.hours, initialize= demand)
                                model.pv_production=        pe.Param(model.hours, initialize= pv_production)

                                model.Efficiency_charge=    pe.Param(model.hours, initialize= efficiency_charge)
                                model.Efficiency_inverter=  pe.Param(model.hours, initialize= efficiency_discharge)
                                model.Efficiency_discharge= pe.Param(model.hours, initialize= efficiency_inverter)

                                model.SoCmin=               pe.Param(initialize= (e_bess*0.1))
                                model.SoCmax=               pe.Param(initialize= (e_bess*0.9))

                                model.MaxCharge=            pe.Param(initialize= p_bess)

                                ## OBS!: Read the last value of SoC, OR initial if first day
                                if iteration == 1:
                                    model.SoCinitial=       pe.Param(initialize= soc_initial)
                                #model.SoCinitial=       pe.Param(initialize= soc_initial)
                                else:
                                    if pd.isna(results['SoC'].iloc[len(results) -1]):# == True:
                                        model.SoCinitial=       pe.Param(initialize= (e_bess*0.1))
                                    else:
                                        last_SoC_value=         float(results['SoC'].iloc[-1])
                                        model.SoCinitial=       pe.Param(initialize= last_SoC_value)
                                
                                
                                #model.arbitrageLimit=       pe.Param(model.hours, initialize= (p_solar * 1000000))
                                model.arbitrageLimit=       pe.Param(model.hours, initialize= demand_multiple)
                                if curve == 'Price':
                                    model.costs=            pe.Param(model.flows, model.hours, initialize=costs_economic, default= 1)
                                elif curve == 'CO_2_eq':
                                    model.costs=            pe.Param(model.flows, model.hours, initialize=costs_environmental, default= 1)

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Variables
                                # Variable for the power flows in our system
                                model.p=            pe.Var(model.flows, model.hours, domain= pe.NonNegativeReals)
                                # Variable to track the State-of-Charge in the battery
                                model.SoC=          pe.Var(model.hours, domain= pe.Reals, bounds= (model.SoCmin, model.SoCmax))
                                # Two variables to track/limit the charge/discharge operation at the same time (ref: doi.org/10.2991/978-94-6463-156-2_13)
                                model.Sw_charge=    pe.Var(model.hours, domain= pe.Binary)
                                model.Sw_discharge= pe.Var(model.hours, domain= pe.Binary)

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Objective
                                minCosts = sum(model.p[f, t] * model.costs[f, t] for f in model.flows for t in model.hours)
                                model.objective= pe.Objective(sense= pe.minimize, expr= minCosts)

                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Constraints - Declaration
                                
                                # Load Fullfilment from the flows _to_Load in the system 
                                loadFullfilment=    {t: (model.p['P_PV_to_Load', t] * efficiency_inverter + model.p['P_BESS_to_Load', t] * (efficiency_discharge * efficiency_inverter) + model.p['P_Grid_to_Load', t]) == model.demand[t] for t in model.hours}

                                # Allocation of PV production as a resource
                                pvProduction=       {t: (model.p['P_PV_to_Load', t] + model.p['P_PV_to_BESS', t] + model.p['P_PV_to_Grid', t] + model.p['P_PV_curtailment', t]) == model.pv_production[t] for t in model.hours}
                                
                                # Load multiple limits the arbitrage flow
                                arbitrageFlow=      {t: (model.p['P_BESS_to_Grid', t] * (efficiency_discharge * efficiency_inverter) + model.p['P_PV_to_Grid', t] * efficiency_inverter) <= model.arbitrageLimit[t] for t in model.hours}
                                
                                # Limit to grid connection - for the purposes at this point it 1 GW (meaning multiple times higher than our system, representing an "unlimited" amount of power we can draw from the grid)
                                gridFlow=           {t: (model.p['P_Grid_to_Load', t] + model.p['P_Grid_to_BESS', t]) <= model.grid_production for t in model.hours}
                                
                                # Constraint to limit maximum charging capacity
                                bessChargeFlow=     {t: (model.p['P_Grid_to_BESS', t] + model.p['P_PV_to_BESS', t]) <= model.Sw_charge[t] * model.MaxCharge for t in model.hours}
                                bessDischargeFlow=  {t: (model.p['P_BESS_to_Grid', t] + model.p['P_BESS_to_Load', t]) <= model.Sw_discharge[t] * model.MaxCharge for t in model.hours}
                                
                                # Constraint to limit charging/discharging in the same time block
                                # This can be implemented by adjusting the bessChargeFlow and bessDischargeFlow (see: doi.org/10.2991/978-94-6463-156-2_13)
                                limitValidation=    {t: model.Sw_charge[t] + model.Sw_discharge[t] <= 1 for t in model.hours}


                                def storage_state(model, t):
                                            if t == model.hours.first():
                                                return model.SoC[t] == model.SoCinitial + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (efficiency_discharge * efficiency_inverter))
                                            else:
                                                return model.SoC[t] == model.SoC[t-1] + ((model.p['P_PV_to_BESS', t] + model.p['P_Grid_to_BESS', t]) * (efficiency_charge * efficiency_inverter)) - ((model.p['P_BESS_to_Load', t] + model.p['P_BESS_to_Grid', t])/ (efficiency_discharge * efficiency_inverter))

                                ## Constraints - Implementation
                                model.demandRule=       pe.Constraint(model.hours, expr= loadFullfilment)
                                model.pvProductionRule= pe.Constraint(model.hours, expr= pvProduction)
                                model.arbitrageRule=    pe.Constraint(model.hours, expr= arbitrageFlow)
                                model.gridRule=         pe.Constraint(model.hours, expr= gridFlow)
                                model.chargeRule=       pe.Constraint(model.hours, expr= bessChargeFlow)
                                model.dischargeRule=    pe.Constraint(model.hours, expr= bessDischargeFlow)
                                #model.limitCharge=      pe.Constraint(model.hours, expr= limitCharge)
                                #model.limitDischarge=   pe.Constraint(model.hours, expr= limitDischarge)
                                model.limitValidation=  pe.Constraint(model.hours, expr= limitValidation)
                                model.charge_state=     pe.Constraint(model.hours, expr = storage_state)


                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Solve the model for the time window (iteration)
                                solver= po.SolverFactory('glpk')
                                ## What happened often is that the Long-Step Dual Simplex method is used, which uses an iterative approach to arrive at a "local globally-optimal" solution, which can take a very long time to compute
                                ## We will limit the maximum time for finding the optimal solution, based either on the time limit (in seconds) or the minimal optimality gap (in percentages)
                                ## The two lines below set the time limit to 18 seconds and the minimal optimality gap to 3% (or lower)
                                solver.options['tmlim']= 15
                                solver.options['mipgap']= 0.05
                                ## Set tee to True if you want to see what the solver is doing (for troubleshooting etc.)
                                modelresults= solver.solve(model, tee= False)





                                ## -----------------------------------------------------------------------------------------------------------------------------------------------------------
                                ## Results Handling
                                for flow in flows_results:
                                    for hour in hours:
                                        if flow == 'SoC':
                                            results['SoC'][hour]= model.SoC[hour].value
                                        else:
                                            results[flow][hour]= model.p[flow, hour].value
                                    results[flow] = results[flow].astype(float)
                                # results= results.reset_index()
                                # results= results.rename(columns={'index': 'Hour'})
                                
                                #print(type(float(results['SoC'].iloc[-1])))
                                #print(results['SoC'].iloc[len(results) -1])

                                results['sum_power_flows'] = results.P_PV_to_Load + results.P_BESS_to_Load + results.P_Grid_to_Load
                                results['sum_power_flows'] = results['sum_power_flows'].astype(float)

                                for param in model.component_data_objects(pe.Param, active= True):
                                    model.del_component(param)
                                for variable in model.component_data_objects(pe.Var, active= True):
                                    model.del_component(variable)
                                for cons in model.component_data_objects(pe.Constraint, active=True):
                                    model.del_component(cons)
                                for obj in model.component_data_objects(pe.Objective, active= True):
                                    model.del_component(obj)
                                #model.del_component(model.SoCinitial)
                                #model.del_component(model.charge_state)
                                model.del_component(model.hours)
                                model.del_component(model.flows)


                                
                            results= results.reset_index()
                            results= results.rename(columns={'index': 'Hour'})
                            results.to_csv(
                                f'RESULTS/{curve}_{region}_{load_profile}_{solar_multiple}pv_{bess_multiple}bess_{bess_capacity}hr.csv',
                                sep= ',',
                                index=False,
                                columns=None
                            )
                            # if curve == 'price':
                            #     results.to_csv(
                            #         f'Results/Batch5/CaseB/price_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv',
                            #         sep= ',',
                            #         index=False,
                            #         columns=None
                            #     )
                            # elif curve == 'co2':
                            #     results.to_csv(
                            #         f'Results/Batch5/CaseB/co2_{int(solar_multiple)}_{bess_multiple}_{t_bess}.csv',
                            #         sep= ',',
                            #         index=False,
                            #         columns=None
                            #     )
                            results = []
                            end_time= time.time()
                            execution_time= end_time - start_time

                            # Convert execution time to hours, minutes, and seconds
                            hours, remainder = divmod(execution_time, 3600)
                            minutes, seconds = divmod(remainder, 60)

                            # Print the execution time in the format hh:mm:ss
                            print("Execution time: {:02}:{:02}:{:02}".format(int(hours), int(minutes), int(seconds)))
                            print('-' * 50)
                            #print(' ')
#results